<a href="https://colab.research.google.com/github/Johnny880724/AFSA_material/blob/main/fish_on_real_crystal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
# Step 1: Mount Google Drive to access your files
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Import necessary libraries
import numpy as np

# Step 3: Define the path to your .txt file
# You can find the path after mounting your Google Drive.
# Make sure to adjust the path based on where your file is located in Drive.
file_path = '/content/drive/MyDrive/crystal data/5A_parsed_file.txt'

# Step 4: Load the .txt file into a NumPy array
# Assuming the file is space-separated, but adjust delimiter if needed
matrix = np.loadtxt(file_path)
matrix = matrix.T
# Step 5: Display the matrix

# print("Matrix Loaded from File:")

print(np.shape(matrix))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
(10, 594)


In [20]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C

def fit_gaussian_process(input_array):
    """
    Fit a Gaussian Process model to the provided n-dimensional array.

    Args:
    - input_array (np.ndarray): The n-dimensional array where the first row contains energy (z-values)
                                and the subsequent rows represent the coordinates in n-dimensional space.

    Returns:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    """
    # Separate the energy (first row) and location data (other rows)
    energy_values = input_array[0, :]  # Energy values (z)
    location_data = input_array[1:, :].T  # Location data (x, y, ...)

    # Define the kernel: we use a product of constant kernel and RBF kernel (Gaussian kernel)
    kernel = C(1.0, (1e-4, 1e1)) * RBF(1.0, (1e-4, 1e1))  # Constant * RBF kernel

    # Initialize the GaussianProcessRegressor with the kernel
    gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

    # Fit the GP model to the data
    gp.fit(location_data, energy_values)

    return gp

def predict_energy(gp, location):
    """
    Predict the energy at a given location using the fitted Gaussian Process model.

    Args:
    - gp (GaussianProcessRegressor): The trained Gaussian Process model.
    - location (np.ndarray): The location for which to predict the energy.

    Returns:
    - energy (float): The predicted energy at the given location.
    """
    # Ensure the location is in the right shape (1 x n_dim)
    location = np.array(location).reshape(1, -1)

    # Use the Gaussian Process model to predict the energy at the specified location
    energy, _ = gp.predict(location, return_std=True)

    return energy[0]  # Return the predicted energy value

# Example Usage:
# Input: An example n-dimensional array where the first row is energy, and others are coordinates.
input_array = np.array([
    [1.0, 2.0, 3.0, 4.0],  # Energy (z-values)
    [0.0, 1.0, 2.0, 3.0],  # Location 1 (x)
    [1.0, 1.5, 2.0, 2.5],  # Location 2 (y)
])

# Fit the Gaussian Process model to the input data
gp_model = fit_gaussian_process(matrix)

# Predict the energy at a new location (e.g., at location [2.5, 2.0])
new_location = [1,1,1,0,0,0,1,0,1]
predicted_energy = predict_energy(gp_model, new_location)

# Output the predicted energy
print(f"Predicted energy at location {new_location}: {predicted_energy}")


Predicted energy at location [1, 1, 1, 0, 0, 0, 1, 0, 1]: -0.0012442610899849132


/usr/local/lib/python3.10/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__constant_value is close to the specified upper bound 10.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(
